In [16]:
import pandas as pd
import numpy as np
import os
os.listdir()
from sklearn.preprocessing import OneHotEncoder
import typing

In [17]:
df = pd.read_csv("medical_students_dataset.csv")
##https://www.kaggle.com/datasets/slmsshk/medical-students-dataset

In [18]:
df = df.dropna()
df.index = [i for i in range(len(df.index))]
df = df.iloc[:, 1:]

In [19]:
df ##  some preprocessing here!!!!  to fit the things into 

,Age,Gender,Height,Weight,Blood Type,BMI,Temperature,Heart Rate,Blood Pressure,Cholesterol,Diabetes,Smoking
0,32.0,Female,182.537664,55.741083,A,16.729017,98.260293,76.0,130.0,216.0,Yes,No
1,28.0,Male,186.489402,52.389752,AB,15.063921,98.227788,85.0,123.0,128.0,No,No
2,34.0,Female,182.416302,76.371050,AB,22.950992,98.118274,86.0,97.0,247.0,No,No
3,31.0,Male,166.489929,49.955569,B,18.022207,98.809750,82.0,96.0,223.0,No,No
4,29.0,Female,179.909041,90.679436,AB,28.015787,98.782269,81.0,108.0,227.0,No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...
50788,25.0,Female,156.297842,90.690186,B,37.123963,98.664591,70.0,132.0,164.0,Yes,No
50789,23.0,Female,162.884951,82.485778,B,31.089745,98.852347,65.0,94.0,188.0,No,Yes
50790,34.0,Male,165.651315,93.099756,A,33.928040,97.862209,62.0,100.0,205.0,No,No
50791,34.0,Male,161.590030,90.877589,B,34.803881,98.728836,70.0,96.0,208.0,No,No


In [20]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
non_numeric = ["Gender", "Blood Type", "Diabetes", "Smoking"]
numeric = {"Age":0, "Height":0, "Weight":0, "BMI":0, "Temperature":0, "Heart Rate":0, "Blood Pressure":0, "Cholesterol":0}
one_hotted = ["Blood Type"]

In [21]:
for feature in non_numeric:
    df[feature]= LabelEncoder().fit_transform(df[feature])

for feature in numeric.keys():
    mean = df[feature].mean()
    std = df[feature].std()
    numeric[feature] = mean, std
    df[feature] = (df[feature] - mean)/(std + 1e-5) 

In [22]:
df = df.drop("Blood Type", axis = 1)
#df_onehotted = OneHotEncoder(sparse_output=False).fit_transform(np.array(df["Blood Type"]).reshape(-1,1))

In [23]:
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch import nn as nn

In [24]:
def return_splitted(df:pd.DataFrame)->tuple[np.ndarray,]:
    df = df.sample(frac = 1)
    N = int(len(df)*0.8)
    train, test = map(np.array, [df.iloc[:N,:], df.iloc[N:, :]])
    return train, test
train, test = return_splitted(df)

In [25]:
import auto_encoder
from auto_encoder import auto_encoder_bb
from typing import Callable

In [ ]:
df

In [ ]:
#auto_encoder_bb([20, 50, 4])(torch.randn(10, 20), encoder_output = True)
F.one_hot(torch.tensor([0, 3, 1]), 5).shape

In [26]:
def loss(numerical_columns:list[int] = [], 
         binary_columns:list[int] = [],
         categorical_columns:list[tuple[float,float]] = [],
         )->Callable[[torch.Tensor, torch.Tensor],torch.Tensor]:
        #@torch.compile ## since we have a lot of for loops compiling may save some time!!!
        def temp_loss(X:torch.Tensor, y:torch.Tensor)->torch.Tensor:
            loss = nn.MSELoss()(X[:, numerical_columns], y[:, numerical_columns])
            for feature in categorical_columns:
                i, class_size = feature
                loss += F.cross_entropy(X[:, i:i+class_size], y[:, i:i+class_size])
            for i in binary_columns:
                loss += F.binary_cross_entropy_with_logits(X[:, i], y[:, i])               
            return loss
        return temp_loss


In [27]:
loss_fn = loss(numerical_columns = [0,2,3,4,5,6,7,8], binary_columns = [1,9,10])

In [28]:
torch_df = torch.tensor(df.to_numpy(), dtype = torch.float32)

In [29]:
model = auto_encoder_bb([11, 10, 9, 8, 5])

In [30]:
from torch.optim import SGD, Adam

In [31]:
optimizer = Adam(model.parameters())
for i in range(10000):
    optimizer.zero_grad()
    y_pred = model(torch_df)
    loss_ = loss_fn(y_pred, torch_df)
    loss_.backward()
    optimizer.step()
    if i % 100 == 0:
        print(f"The loss is now {loss_.item()}")    

The loss is now 3.071112632751465
The loss is now 2.8233225345611572
The loss is now 2.4975943565368652
The loss is now 2.404484748840332
The loss is now 2.372570514678955
The loss is now 2.3289947509765625
The loss is now 2.3010337352752686
The loss is now 2.2798378467559814
The loss is now 2.267202138900757
The loss is now 2.260279417037964
The loss is now 2.2562623023986816
The loss is now 2.2529959678649902


KeyboardInterrupt: 

In [ ]:
model(torch_df[6, :], True)

In [ ]:
torch_df[3, :]

In [ ]:
model(torch_df[3, :])